# Creating Pages

This notebook covers creating pages in Notion with different property types and configurations:
- Creating pages in databases (as data source rows)
- Creating child pages under other pages
- Setting properties (title, select, number, date, checkbox, etc.)
- Adding icons and covers
- Creating pages with content blocks
- Batch page creation patterns

## Prerequisites

Make sure you have set these environment variables:
- `NOTION_API_TOKEN` - Your Notion integration token
- `NOTION_TEST_PAGE_ID` - A test page ID where we can create test pages and databases

## Setup: Load Dependencies and Initialize Client

In [1]:
// Load the Kotlin Notion Client library from Maven Central
@file:DependsOn("it.saabel:kotlin-notion-client:0.1.0")

// Import necessary classes
import it.saabel.kotlinnotionclient.NotionClient
import it.saabel.kotlinnotionclient.models.base.SelectOptionColor
import it.saabel.kotlinnotionclient.models.datasources.SortDirection
import it.saabel.kotlinnotionclient.models.pages.PageProperty
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll

// Initialize the client
val apiToken = System.getenv("NOTION_API_TOKEN")
    ?: error("❌ NOTION_API_TOKEN environment variable not set")

val parentPageId = System.getenv("NOTION_TEST_PAGE_ID")
    ?: error("❌ NOTION_TEST_PAGE_ID environment variable not set")

val notion = NotionClient(apiToken)

println("✅ NotionClient initialized successfully!")

✅ NotionClient initialized successfully!


## Understanding Page Types in Notion

In Notion, there are two main ways to create pages:

1. **Database Pages (Data Source Rows)**: Pages created within a database/data source
   - Have custom properties defined by the database schema
   - Use `parent.dataSource(dataSourceId)` when creating
   - Properties set via the `properties { }` block

2. **Child Pages**: Regular pages created as children of another page
   - Have only the built-in `title` property
   - Use `parent.page(pageId)` when creating
   - Use `title()` directly (not in properties block)
   - Can have content blocks added immediately

## Example 1: Create a Test Database for Our Examples

First, let's create a database with various property types that we'll use for demonstrating page creation.

In [2]:
val database = runBlocking {
    notion.databases.create {
        parent.page(parentPageId)
        title("Project Tasks")

        properties {
            title("Task Name")
            select("Status") {
                option("To Do")
                option("In Progress")
                option("Completed")
            }
            select("Priority") {
                option("Low")
                option("Medium")
                option("High")
            }
            number("Estimate (hours)")
            date("Due Date")
            checkbox("Is Blocking")
            richText("Notes")
        }
    }
}

// Get the data source ID - we'll need this to create pages
val dataSourceId = database.dataSources.firstOrNull()?.id
    ?: error("Database has no data sources")

println("✅ Database created successfully!")
println("   Database ID: ${database.id}")
println("   Data Source ID: $dataSourceId")
println("   Title: ${database.title.firstOrNull()?.plainText}")
println("   URL: ${database.url}")

✅ Database created successfully!
   Database ID: 3e602970-64e9-43d6-a2e1-e32dac494105
   Data Source ID: 1fb8b91f-dee8-48d2-bcf5-4e611a593d50
   Title: Project Tasks
   URL: https://www.notion.so/3e60297064e943d6a2e1e32dac494105


## Example 2: Create a Page in a Data Source (Database Row)

Let's create our first page in the database with various property types.

In [5]:
import it.saabel.kotlinnotionclient.models.pages.PageIcon

val taskPage = runBlocking {
    notion.pages.create {
        // Specify the data source as parent
        parent.dataSource(dataSourceId)

        // Set property values
        properties {
            title("Task Name", "Implement user authentication")
            select("Status", "In Progress")
            select("Priority", "High")
            number("Estimate (hours)", 8.0)
            date("Due Date", "2025-10-20")
            checkbox("Is Blocking", true)
            richText("Notes", "This is a critical feature for the MVP release.")
        }

        // Optional: Set icon and cover
        icon.emoji("🔐")
        cover.external("https://images.unsplash.com/photo-1557683316-973673baf926")
    }
}

println("✅ Page created successfully!")
println("   Page ID: ${taskPage.id}")
println("   URL: ${taskPage.url}")

// Access the properties
val titleProp = taskPage.properties["Task Name"] as? PageProperty.Title
val statusProp = taskPage.properties["Status"] as? PageProperty.Select
val priorityProp = taskPage.properties["Priority"] as? PageProperty.Select

println("   Title: ${titleProp?.plainText}")
println("   Status: ${statusProp?.select?.name}")
println("   Priority: ${priorityProp?.select?.name}")
println("   Icon: ${(taskPage.icon as? PageIcon.Emoji)?.emoji}")

✅ Page created successfully!
   Page ID: 28fc63fd-82ed-8196-a77b-d78388b00349
   URL: https://www.notion.so/Implement-user-authentication-28fc63fd82ed8196a77bd78388b00349
   Title: Implement user authentication
   Status: In Progress
   Priority: High
   Icon: 🔐


## Example 3: Create a Child Page

Now let's create a regular page as a child of our test page.

In [6]:
val childPage = runBlocking {
    notion.pages.create {
        // Specify another page as parent
        parent.page(parentPageId)

        // For child pages, use title() directly (not in properties block)
        title("Meeting Notes - October 2025")

        // Optional: Add content blocks immediately
        content {
            heading1("Key Takeaways")

            bullet("Decision made on authentication architecture")
            bullet("Timeline set for MVP release in Q1 2026")
            bullet("Team will focus on core features first")

            divider()

            heading2("Action Items")

            number("Review security requirements")
            number("Set up CI/CD pipeline")
            number("Schedule next sprint planning")

            divider()

            paragraph("Next meeting scheduled for next week.")
        }

        // Optional: Set icon
        icon.emoji("📝")
    }
}

println("✅ Child page created successfully!")
println("   Page ID: ${childPage.id}")
println("   URL: ${childPage.url}")

val titleProp = childPage.properties["title"] as? PageProperty.Title
println("   Title: ${titleProp?.plainText}")

✅ Child page created successfully!
   Page ID: 28fc63fd-82ed-81be-a754-d602636d10d0
   URL: https://www.notion.so/Meeting-Notes-October-2025-28fc63fd82ed81bea754d602636d10d0
   Title: Meeting Notes - October 2025


## Example 4: Create a Page with Rich Content

Let's create a more complex page with formatted content using the Rich Text DSL.

In [7]:
val richPage = runBlocking {
    notion.pages.create {
        parent.page(parentPageId)
        title("Project Plan")

        // Add structured content with rich formatting
        content {
            heading1("Project Overview")

            paragraph {
                text("This project aims to ")
                bold("revolutionize")
                text(" how we handle ")
                italic("user authentication")
                text(" and ")
                code("session management")
                text(".")
            }

            divider()

            heading2("Technical Architecture")

            paragraph {
                text("We'll use ")
                bold("JWT tokens")
                text(" for stateless authentication.")
            }

            bullet("Token expiration: 24 hours")
            bullet("Refresh token rotation enabled")
            bullet("OAuth 2.0 integration for social logins")

            divider()

            heading2("Timeline")

            number("Phase 1: Core authentication (2 weeks)")
            number("Phase 2: OAuth integration (1 week)")
            number("Phase 3: Testing and security audit (1 week)")

            divider()

            callout("⚠️") {
                text("Note: Timeline subject to change based on security audit results.")
            }
        }

        icon.emoji("📋")
    }
}

println("✅ Page with rich content created successfully!")
println("   Page ID: ${richPage.id}")
println("   URL: ${richPage.url}")

✅ Page with rich content created successfully!
   Page ID: 28fc63fd-82ed-8114-9956-e696726a0b0c
   URL: https://www.notion.so/Project-Plan-28fc63fd82ed81149956e696726a0b0c


## Example 5: Batch Create Pages

When you need to create multiple pages, you can use Kotlin's `map` function for sequential creation or coroutines for concurrent creation.

In [8]:
// Define multiple tasks to create
val tasksToCreate = listOf(
    Triple("Set up development environment", "To Do", "Low"),
    Triple("Write API documentation", "To Do", "Medium"),
    Triple("Implement error handling", "In Progress", "High"),
    Triple("Write unit tests", "To Do", "Medium"),
    Triple("Deploy to staging", "To Do", "Low")
)

// Create all pages sequentially
val createdPages = runBlocking {
    tasksToCreate.map { (taskName, status, priority) ->
        notion.pages.create {
            parent.dataSource(dataSourceId)
            properties {
                title("Task Name", taskName)
                select("Status", status)
                select("Priority", priority)
            }
        }
    }
}

println("✅ Created ${createdPages.size} pages successfully!")
createdPages.forEach { page ->
    val titleProp = page.properties["Task Name"] as? PageProperty.Title
    val statusProp = page.properties["Status"] as? PageProperty.Select
    println("   - ${titleProp?.plainText} [${statusProp?.select?.name}]")
}

✅ Created 5 pages successfully!
   - Set up development environment [To Do]
   - Write API documentation [To Do]
   - Implement error handling [In Progress]
   - Write unit tests [To Do]
   - Deploy to staging [To Do]


## Example 6: Create Pages with Different Property Types

Let's explore more property types available in Notion.

In [9]:
val propertyExamplePage = runBlocking {
    notion.pages.create {
        parent.dataSource(dataSourceId)

        properties {
            // Title property (required for data source pages)
            title("Task Name", "Property Types Example")

            // Select property (single option)
            select("Status", "To Do")
            select("Priority", "Medium")

            // Number property
            number("Estimate (hours)", 12.5)

            // Date property (just a date)
            date("Due Date", "2025-11-01")

            // Checkbox property
            checkbox("Is Blocking", false)

            // Rich text property
            richText("Notes", "This page demonstrates various property types we can set when creating pages.")
        }

        icon.emoji("🎯")
    }
}

println("✅ Page with various property types created!")
println("   Page ID: ${propertyExamplePage.id}")

// Access different property types
val titleProp = propertyExamplePage.properties["Task Name"] as? PageProperty.Title
val numberProp = propertyExamplePage.properties["Estimate (hours)"] as? PageProperty.Number
val dateProp = propertyExamplePage.properties["Due Date"] as? PageProperty.Date
val checkboxProp = propertyExamplePage.properties["Is Blocking"] as? PageProperty.Checkbox

println("   Title: ${titleProp?.plainText}")
println("   Estimate: ${numberProp?.number} hours")
println("   Due Date: ${dateProp?.date?.start}")
println("   Is Blocking: ${checkboxProp?.checkbox}")

✅ Page with various property types created!
   Page ID: 28fc63fd-82ed-8149-b2e1-db7aa82dc776
   Title: Property Types Example
   Estimate: 12.5 hours
   Due Date: 2025-11-01
   Is Blocking: false


## Example 7: Working with Icons and Covers

Pages can have icons (emoji or external images) and cover images.

In [10]:
import it.saabel.kotlinnotionclient.models.pages.PageCover

val styledPage = runBlocking {
    notion.pages.create {
        parent.dataSource(dataSourceId)

        properties {
            title("Task Name", "Design new landing page")
            select("Status", "In Progress")
            select("Priority", "High")
        }

        // Set an emoji icon
        icon.emoji("🎨")

        // Set a cover image from Unsplash
        cover.external("https://images.unsplash.com/photo-1558618666-fcd25c85cd64")
    }
}

println("✅ Page with icon and cover created!")
println("   Page ID: ${styledPage.id}")
println("   Icon: ${(styledPage.icon as? PageIcon.Emoji)?.emoji}")
println("   Cover URL: ${(styledPage.cover as? PageCover.External)?.external?.url}")

✅ Page with icon and cover created!
   Page ID: 28fc63fd-82ed-8159-a27b-f9292a49499d
   Icon: 🎨
   Cover URL: https://images.unsplash.com/photo-1558618666-fcd25c85cd64


## Cleanup

Let's clean up by archiving the test database and child pages we created.

In [11]:
runBlocking {
    // Archive the test database (this will archive all pages in it)
    notion.databases.archive(database.id)

    // Archive the child pages we created
    notion.pages.archive(childPage.id)
    notion.pages.archive(richPage.id)
}

println("✅ Test data archived successfully!")

✅ Test data archived successfully!


## Next Steps

Now that you understand creating pages, explore:
- **[04-working-with-blocks.ipynb](./04-working-with-blocks.ipynb)** - Build page content with blocks
- **[05-rich-text-dsl.ipynb](./05-rich-text-dsl.ipynb)** - Format text with the Rich Text DSL
- **[06-advanced-queries.ipynb](./06-advanced-queries.ipynb)** - Advanced filtering and pagination

## Key Takeaways

- **Two Page Types**: Database pages (data source rows) vs child pages
- **Database Pages**: Use `parent.dataSource(id)` and set properties via `properties { }` block
- **Child Pages**: Use `parent.page(id)` and set title with `title()` directly
- **Property Types**: Support for title, select, number, date, checkbox, rich text, and more
- **Content Blocks**: Can add content immediately when creating pages with `content { }` block
- **Icons & Covers**: Customize page appearance with emojis or external images
- **Batch Creation**: Use `map` for sequential or coroutines (`async`/`awaitAll`) for concurrent creation
- **Type-Safe Access**: Use pattern matching with `as?` to safely access property values

## Bonus: Understanding the Differences

Here's a quick reference for the key differences between the two page creation patterns:

| Aspect | Database Page | Child Page |
|--------|---------------|------------|
| **Parent** | `parent.dataSource(id)` | `parent.page(id)` |
| **Title** | `properties { title("PropName", "value") }` | `title("value")` |
| **Custom Properties** | ✅ Yes (defined by database schema) | ❌ No (only built-in title) |
| **Content Blocks** | ✅ Yes (via `content { }`) | ✅ Yes (via `content { }`) |
| **Icons & Covers** | ✅ Yes | ✅ Yes |
| **Use Case** | Structured data (tasks, contacts, etc.) | Documentation, notes, wikis |